In [2]:
import json

In [45]:
file_name = './Books_small_10000.json' 

#### accessing our file and openeing it 

In [46]:
with open(file_name) as f:
    for line in f:
        print(line)
        break

{"reviewerID": "A1F2H80A1ZNN1N", "asin": "B00GDM3NQC", "reviewerName": "Connie Correll", "helpful": [0, 0], "reviewText": "I bought both boxed sets, books 1-5.  Really a great series!  Start book 1 three weeks ago and just finished book 5.  Sloane Monroe is a great character and being able to follow her through both private life and her PI life gets a reader very involved!  Although clues may be right in front of the reader, there are twists and turns that keep one guessing until the last page!  These are books you won't be disappointed with.", "overall": 5.0, "summary": "Can't stop reading!", "unixReviewTime": 1390435200, "reviewTime": "01 23, 2014"}



##### Getting the reviewText and the Overall
we'll used json.loads to access the key value pairs in the json object

In [47]:
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        print(review['reviewText'])
        print(review['overall'])
        break

I bought both boxed sets, books 1-5.  Really a great series!  Start book 1 three weeks ago and just finished book 5.  Sloane Monroe is a great character and being able to follow her through both private life and her PI life gets a reader very involved!  Although clues may be right in front of the reader, there are twists and turns that keep one guessing until the last page!  These are books you won't be disappointed with.
5.0


#### Putting the review text and scores in an array 

In [48]:
reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append((review['reviewText'],review['overall']))
        
reviews[6][1]
        

2.0

#### FOR BETTER READABLITY, LETS CREATE A CLASS FOR REVIEW

In [98]:
import random
class Sentiment:
    POSITIVE = "POSITIVE"
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"


class Review:
    def __init__(self,text,score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
     
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
        
        
        
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews
        
    def evenly_distribute(self):
        negative = list(filter(lambda x:x.sentiment == Sentiment.NEGATIVE,self.reviews))
        positive = list(filter(lambda x:x.sentiment == Sentiment.POSITIVE,self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        
     
    
   

In [50]:
reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'],review['overall']))
        
print(reviews[29].text)
reviews[29].sentiment



The saga continues at the same high quality level as before.  Some predictable and some twists and turns from left field. Look forward to reading the next book.


'POSITIVE'

#### PREP OUR DATA 

In [51]:
from sklearn.model_selection import train_test_split


In [99]:
training,test = train_test_split(reviews,test_size=0.33,random_state=42)
cont = ReviewContainer(training)
cont.evenly_distribute()
len(cont.reviews)

872

In [53]:
len(training)

6700

In [54]:
training[0].text

"Olivia Hampton arrives at the Dunraven family home as cataloger of their extensive library. What she doesn't expect is a broken carriage wheel on the way. Nor a young girl whose mind is clearly gone, an old man in need of care himself (and doesn&#8217;t quite seem all there in Olivia&#8217;s opinion). Furthermore, Marion Dunraven, the only sane one of the bunch and the one Olivia is inexplicable drawn to, seems captive to everyone in the dusty old house. More importantly, she doesn't expect to fall in love with Dunraven's daughter Marion.Can Olivia truly believe the stories of sadness and death that surround the house, or are they all just local neighborhood rumor?Was that carriage trouble just a coincidence or a supernatural sign to stay away? If she remains, will the Castle&#8217;s dark shadows take Olivia down with them or will she and Marion long enough to declare their love?Patty G. Henderson has created an atmospheric and intriguing story in her Gothic tale. I found this to be a

##### since we want to be able to predict our sentiment for every text input, so we can split the training array into train_x for text and train_y for the sentiment

In [55]:
train_x = [x.text for x in training]
train_y = [x.sentiment for x in training]
test_x = [x.text for x in test]
test_y = [x.sentiment for x in test]

In [56]:
train_x[3]

'It was just one of those books that never went anywhere. I like books that get your attention in the beginning and not drag out until a quarter way through. I decided to give it an early death - delete!'

In [57]:
train_y[3]

'NEGATIVE'

In [58]:
len(test_x)

3300

#### BAG OF WORDS VECTORIZATION
we'll be using bag of words to convert our train array to vectors of 1 and 0 for machine learning 

In [59]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)



In [60]:
train_x_vectors[0]

<1x26615 sparse matrix of type '<class 'numpy.int64'>'
	with 131 stored elements in Compressed Sparse Row format>

In [61]:
print(train_x[3])
print(train_x_vectors[3].toarray())

It was just one of those books that never went anywhere. I like books that get your attention in the beginning and not drag out until a quarter way through. I decided to give it an early death - delete!
[[0 0 0 ... 0 0 0]]


##### now we have our X and Y in a proper format, let's do some classification

### Classification 

#### Linear SVM 

In [62]:
from sklearn import svm

Let test our classifier by predicting the sentiment for a particular text

In [63]:
clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors,train_y)
clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

#### USING DECISION TREE

In [64]:
from sklearn.tree import DecisionTreeClassifier

In [65]:
clf_dec = DecisionTreeClassifier()

In [66]:
clf_dec.fit(train_x_vectors,train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [67]:
clf_dec.predict(test_x_vectors[3])

array(['NEUTRAL'], dtype='<U8')

#### USING NAIVE BAYES

In [68]:
from sklearn.naive_bayes import GaussianNB

In [69]:
clf_gnb = GaussianNB()

In [70]:
clf_gnb.fit((train_x_vectors).toarray(),train_y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [71]:
clf_gnb.predict((test_x_vectors[3]).toarray())

array(['POSITIVE'], dtype='<U8')

#### USING LOGISTIC REGRESSION 

In [72]:
from sklearn.linear_model import LogisticRegression


In [73]:
clf_lgr = LogisticRegression()
clf_lgr.fit(train_x_vectors,train_y)

C:\Users\Mac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mac\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [74]:
clf_lgr.predict(test_x_vectors[3])

array(['POSITIVE'], dtype='<U8')

#### EVALUATION 
mean accuracy for all our models 

In [75]:
clf_svm.score(test_x_vectors,test_y)

0.8124242424242424

In [76]:
clf_dec.score(test_x_vectors,test_y)

0.7684848484848484

In [77]:
clf_gnb.score((test_x_vectors).toarray(),test_y)

0.6587878787878788

In [78]:
clf_lgr.score(test_x_vectors,test_y)

0.8478787878787879

#### F1 SCORE 

As a data scientist, when doing classification problems, you'll need to pay attention to the F1 score for accuracy of your model

In [79]:
from sklearn.metrics import f1_score

In [80]:
f1_score(test_y,clf_svm.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE,Sentiment.NEUTRAL])

array([0.90738061, 0.40268456, 0.2656    ])

In [81]:
f1_score(test_y,clf_gnb.predict(test_x_vectors.toarray()),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE,Sentiment.NEUTRAL])

array([0.7996939 , 0.11851852, 0.1260745 ])

In [82]:
f1_score(test_y,clf_dec.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE,Sentiment.NEUTRAL])

array([0.87475573, 0.16541353, 0.14335664])

In [83]:
f1_score(test_y,clf_lgr.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE,Sentiment.NEUTRAL])

array([0.92493017, 0.4092219 , 0.29714286])

In [85]:
train_y.count(Sentiment.NEGATIVE)

436

#### WE CAN SEE THAT THE FOLLOWING SCORES ARE HIGH ON POSITIVE BUT QUITE LOW ON THE OTHERS, SO WE WILL REDUCE OUR LABELS TO JUST POSITIVE AND NEGATIVE AND TRY TO WORK ON THE DATA